In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from dasly import utils

In [2]:
log = pd.read_excel('../data/logs/vehicle-logs.xlsx')
log

,Time,Type,Trailer,Direction camera,Direction map,Note,YOLO,No signal
0,2023-10-05 08:06:55.000,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN
1,2023-10-05 08:07:39.000,Truck,NaN,going far,Trondheim -> Åstfjordbrua,NaN,NaN,NaN
2,2023-10-05 08:08:02.000,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN
3,2023-10-05 08:08:12.000,Car,NaN,going far,Trondheim -> Åstfjordbrua,NaN,NaN,NaN
4,2023-10-05 08:08:25.000,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...
351,2023-10-05 11:03:56.070,Car,NaN,going far,Trondheim -> Åstfjordbrua,NaN,NaN,NaN
352,2023-10-05 11:04:22.070,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN
353,2023-10-05 11:04:53.260,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN
354,2023-10-05 11:05:20.580,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN


In [23]:
(
    log
    .loc[lambda df: df['No signal'] != 1]
    .groupby('Type')
    .size()
)

Type
Bus        3
Car      229
Moto       3
Other      1
Truck     68
Van       42
dtype: int64

In [24]:
(
    log
    .loc[lambda df: df['No signal'] != 1]
    .loc[lambda df: df['Type'].isin(['Car', 'Bus', 'Truck', 'Van'])]
    .assign(Type=lambda df: np.where(df['Type'].isin(['Car', 'Van']), 'Small', 'Large'))
    .groupby('Type')
    .size()
)

Type
Large     71
Small    271
dtype: int64

In [25]:
(
    log
    .loc[lambda df: df['No signal'] != 1]
    .loc[lambda df: df['Type'].isin(['Car', 'Bus', 'Truck', 'Van'])]
    .assign(Type2=lambda df: np.where(df['Type'].isin(['Car', 'Van']), 'Small', 'Large'))
    .assign(Time2=lambda df: np.where(
        df['Direction camera'] == 'coming close',
        df['Time'] - pd.Timedelta(seconds=30),
        df['Time'] + pd.Timedelta(seconds=30)
    ))
)

,Time,Type,Trailer,Direction camera,Direction map,Note,YOLO,No signal,Type2,Time2
0,2023-10-05 08:06:55.000,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN,Small,2023-10-05 08:06:25.000
1,2023-10-05 08:07:39.000,Truck,NaN,going far,Trondheim -> Åstfjordbrua,NaN,NaN,NaN,Large,2023-10-05 08:08:09.000
2,2023-10-05 08:08:02.000,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN,Small,2023-10-05 08:07:32.000
3,2023-10-05 08:08:12.000,Car,NaN,going far,Trondheim -> Åstfjordbrua,NaN,NaN,NaN,Small,2023-10-05 08:08:42.000
5,2023-10-05 08:13:24.000,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN,Small,2023-10-05 08:12:54.000
...,...,...,...,...,...,...,...,...,...,...
351,2023-10-05 11:03:56.070,Car,NaN,going far,Trondheim -> Åstfjordbrua,NaN,NaN,NaN,Small,2023-10-05 11:04:26.070
352,2023-10-05 11:04:22.070,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN,Small,2023-10-05 11:03:52.070
353,2023-10-05 11:04:53.260,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN,Small,2023-10-05 11:04:23.260
354,2023-10-05 11:05:20.580,Car,NaN,coming close,Åstfjordbrua -> Trondheim,NaN,NaN,NaN,Small,2023-10-05 11:04:50.580


In [26]:
time_ends = (
    log
    .loc[lambda df: df['No signal'] != 1]
    .loc[lambda df: df['Type'].isin(['Car', 'Bus', 'Truck', 'Van'])]
    .assign(Type=lambda df: np.where(df['Type'].isin(['Car', 'Van']), 'Small', 'Large'))
    .assign(Time2=lambda df: np.where(
        df['Direction camera'] == 'coming close',
        df['Time'] - pd.Timedelta(seconds=30),
        df['Time'] + pd.Timedelta(seconds=30)
    ))
    .loc[:, ['Time', 'Time2']]
    .to_numpy()
)

time_ends_df = pd.DataFrame(np.sort(time_ends, axis=1), columns=['t1', 't2'])
time_ends_df

,t1,t2
0,2023-10-05 08:06:25.000,2023-10-05 08:06:55.000
1,2023-10-05 08:07:39.000,2023-10-05 08:08:09.000
2,2023-10-05 08:07:32.000,2023-10-05 08:08:02.000
3,2023-10-05 08:08:12.000,2023-10-05 08:08:42.000
4,2023-10-05 08:12:54.000,2023-10-05 08:13:24.000
...,...,...
337,2023-10-05 11:03:56.070,2023-10-05 11:04:26.070
338,2023-10-05 11:03:52.070,2023-10-05 11:04:22.070
339,2023-10-05 11:04:23.260,2023-10-05 11:04:53.260
340,2023-10-05 11:04:50.580,2023-10-05 11:05:20.580


In [28]:
vehicle_counts = (
    log
    .loc[lambda df: df['No signal'] != 1]
    .loc[lambda df: df['Type'].isin(['Car', 'Bus', 'Truck', 'Van'])]
    .assign(Type2=lambda df: np.where(df['Type'].isin(['Car', 'Van']), 'Small', 'Large'))
    .assign(Time2=lambda df: np.where(
        df['Direction camera'] == 'coming close',
        df['Time'] - pd.Timedelta(seconds=30),
        df['Time'] + pd.Timedelta(seconds=30)
    ))
    .loc[:, ['Type', 'Type2', 'Direction camera']]
    .reset_index(drop=True)
)
vehicle_counts

,Type,Type2,Direction camera
0,Car,Small,coming close
1,Truck,Large,going far
2,Car,Small,coming close
3,Car,Small,going far
4,Car,Small,coming close
...,...,...,...
337,Car,Small,going far
338,Car,Small,coming close
339,Car,Small,coming close
340,Car,Small,coming close


In [33]:
result = (
    pd.concat([vehicle_counts, time_ends_df], axis=1)
    .assign(s1=lambda df: np.where(df['Direction camera'] == 'coming close', 0, 692))
    .assign(s2=lambda df: np.where(df['Direction camera'] == 'going far', 0, 692))
)
result

,Type,Type2,Direction camera,t1,t2,s1,s2
0,Car,Small,coming close,2023-10-05 08:06:25.000,2023-10-05 08:06:55.000,0,692
1,Truck,Large,going far,2023-10-05 08:07:39.000,2023-10-05 08:08:09.000,692,0
2,Car,Small,coming close,2023-10-05 08:07:32.000,2023-10-05 08:08:02.000,0,692
3,Car,Small,going far,2023-10-05 08:08:12.000,2023-10-05 08:08:42.000,692,0
4,Car,Small,coming close,2023-10-05 08:12:54.000,2023-10-05 08:13:24.000,0,692
...,...,...,...,...,...,...,...
337,Car,Small,going far,2023-10-05 11:03:56.070,2023-10-05 11:04:26.070,692,0
338,Car,Small,coming close,2023-10-05 11:03:52.070,2023-10-05 11:04:22.070,0,692
339,Car,Small,coming close,2023-10-05 11:04:23.260,2023-10-05 11:04:53.260,0,692
340,Car,Small,coming close,2023-10-05 11:04:50.580,2023-10-05 11:05:20.580,0,692


In [34]:
result.to_excel('../data/logs/vehicle-log-clean.xlsx', index=False)